# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [13]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [14]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [15]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


In [16]:
neighborhoods_coordinates_df = pd.read_csv('neighborhoods_coordinates.csv')
neighborhoods_coordinates_df

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050
...,...,...,...
68,West Portal,37.740260,-122.463880
69,Western Addition,37.792980,-122.435790
70,Westwood Highlands,37.734700,-122.456854
71,Westwood Park,37.734150,-122.457000


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [23]:
# # Define Panel Visualization Functions
# def housing_units_per_year(housing_units):
#     number_of_units_df = sfo_data.groupby('year').mean()
#     number_of_units_df.drop(columns=['gross_rent','sale_price_sqr_foot'],inplace=True)
#     return number_of_units_df


# # def average_gross_rent():    
#     avg_ppsf_and_rent = sfo_data.groupby(sfo_data.index).mean()
#     avg_ppsf_and_rent.drop(columns=['housing_units'],inplace=True)
#     avg_ppsf_and_rent.columns = ['avg_px_sqr_foot','avg_rent']
#     return avg_ppsf_and_rent


def average_sales_price(neighborhood):
#     """Average Sales Price Per Year."""
    average_sales_chart = neighborhood_means_yearly.hvplot.line(y='avg_px_sqr_foot', groupby='neighborhood', widget_location='right_top')
    return average_sales_chart


# def average_price_by_neighborhood():
#     """Average Prices by Neighborhood."""
    
#     neighborhood_means_yearly.hvplot.line(y='avg_px_sqr_foot', groupby='neighborhood', widget_location='right_top')



# def top_most_expensive_neighborhoods():
#     """Top 10 Most Expensive Neighborhoods."""

#     neighborhood_mean = sfo_data.groupby('neighborhood').mean()
#     neighborhood_mean.drop(columns =['housing_units','gross_rent'],inplace=True)
#     neighborhood_mean.sort_values(by='sale_price_sqr_foot',ascending=False,inplace=True)
#     top_10_mean = neighborhood_mean[0:10]
#     top_10_mean
#     top_10_mean.hvplot.bar(rot=45)


# def most_expensive_neighborhoods_rent_sales():
#     """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
#     neighborhood_means_yearly.hvplot.line(y='avg_px_sqr_foot', groupby='neighborhood', widget_location='right_top') + neighborhood_means_yearly.hvplot.line(y='avg_rent', groupby='neighborhood', widget_location='left_top')

    
    
# def parallel_coordinates():
#     """Parallel Coordinates Plot."""

#     # YOUR CODE HERE!



# def parallel_categories():
#     """Parallel Categories Plot."""
    
#     # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [24]:
# Create a Title for the Dashboard
dashboard_title = "# This is my First Dashboard"
row2 = pn.Row(num_foreclosures_plot)

# Create a tab layout for the dashboard
tabs = pn.Tabs(
    ("Average Sales by Neighborhood", average_sales_chart))

tabs


# Create the dashboard

NameError: name 'average_sales_chart' is not defined

## Serve the Panel Dashboard

In [19]:
# Serve the# dashboard
# YOUR CODE HERE!

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()